In [19]:
!pip -q install yellowbrick==1.3

!pip -q install kneed
!pip -q install factor_analyzer
!pip -q install seaborn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer
from yellowbrick.features import PCA as YBPCA
from kneed import KneeLocator
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
#import plotly.express as px

In [21]:
date_eng = pd.read_csv('../feature_engineering/seoulDate_dateEngin.csv')
date_eng

,Unnamed: 0,Unnamed: 0.1,datetime,date,hour,season,holiday,open,rent_count,temperature,...,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week,dayName_of_week,month_name
0,0,1,2017-12-01 00:00:00,2017-12-01,0,Winter,No Holiday,Yes,254,-5.2,...,0.0,0.0,0,12,48,1,335,4,Friday,Dec
1,1,2,2017-12-01 01:00:00,2017-12-01,1,Winter,No Holiday,Yes,204,-5.5,...,0.0,0.0,0,12,48,1,335,4,Friday,Dec
2,2,3,2017-12-01 02:00:00,2017-12-01,2,Winter,No Holiday,Yes,173,-6.0,...,0.0,0.0,0,12,48,1,335,4,Friday,Dec
3,3,4,2017-12-01 03:00:00,2017-12-01,3,Winter,No Holiday,Yes,107,-6.2,...,0.0,0.0,0,12,48,1,335,4,Friday,Dec
4,4,5,2017-12-01 04:00:00,2017-12-01,4,Winter,No Holiday,Yes,78,-6.0,...,0.0,0.0,0,12,48,1,335,4,Friday,Dec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,8756,2018-11-30 19:00:00,2018-11-30,19,Autumn,No Holiday,Yes,1003,4.2,...,0.0,0.0,0,11,48,30,334,4,Friday,Nov
8756,8756,8757,2018-11-30 20:00:00,2018-11-30,20,Autumn,No Holiday,Yes,764,3.4,...,0.0,0.0,0,11,48,30,334,4,Friday,Nov
8757,8757,8758,2018-11-30 21:00:00,2018-11-30,21,Autumn,No Holiday,Yes,694,2.6,...,0.0,0.0,0,11,48,30,334,4,Friday,Nov
8758,8758,8759,2018-11-30 22:00:00,2018-11-30,22,Autumn,No Holiday,Yes,712,2.1,...,0.0,0.0,0,11,48,30,334,4,Friday,Nov


In [22]:
date_eng.describe()

,Unnamed: 0,Unnamed: 0.1,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,4379.500000,4380.500000,11.500000,704.602055,12.882922,58.226256,1.724909,14368.257991,4.073813,0.569111,0.148687,0.750685,6.526027,26.558904,15.720548,183.000000,3.002740
std,2528.938512,2528.938512,6.922582,644.997468,11.944825,20.362413,1.036300,6082.987120,13.060369,0.868746,1.128193,4.367462,3.448048,15.030690,8.796749,105.372043,1.998056
min,0.000000,1.000000,0.000000,0.000000,-17.800000,0.000000,0.000000,270.000000,-30.600000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2189.750000,2190.750000,5.750000,191.000000,3.500000,42.000000,0.900000,9400.000000,-4.700000,0.000000,0.000000,0.000000,4.000000,14.000000,8.000000,92.000000,1.000000
50%,4379.500000,4380.500000,11.500000,504.500000,13.700000,57.000000,1.500000,16980.000000,5.100000,0.010000,0.000000,0.000000,7.000000,27.000000,16.000000,183.000000,3.000000
75%,6569.250000,6570.250000,17.250000,1065.250000,22.500000,74.000000,2.300000,20000.000000,14.800000,0.930000,0.000000,0.000000,10.000000,40.000000,23.000000,274.000000,5.000000
max,8759.000000,8760.000000,23.000000,3556.000000,39.400000,98.000000,7.400000,20000.000000,27.200000,3.520000,35.000000,88.000000,12.000000,52.000000,31.000000,365.000000,6.000000


In [ ]:
date_eng_colnames = list(date_eng.columns)
date_eng_colnames

In [28]:
date_eng_num_colnames = list(date_eng.describe().columns)
date_eng_num_colnames

['Unnamed: 0',
 'Unnamed: 0.1',
 'hour',
 'rent_count',
 'temperature',
 'humidity',
 'wind_speed',
 'visibility',
 'dewpoint_temp',
 'solar_radiation',
 'rainfall',
 'snowfall',
 'month',
 'week',
 'day_of_month',
 'day_of_year',
 'day_of_week']

In [29]:
diff_names = []
for i in date_eng_colnames:
	if i not in date_eng_num_colnames:
		diff_names.append(i)
diff_names

['datetime',
 'date',
 'season',
 'holiday',
 'open',
 'dayName_of_week',
 'month_name']

In [46]:
date_eng2 = date_eng.copy()
date_eng2.drop(columns = diff_names,inplace=True)
date_eng2.head()

,Unnamed: 0,Unnamed: 0.1,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week
0,0,1,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0,12,48,1,335,4
1,1,2,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0,12,48,1,335,4
2,2,3,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0,12,48,1,335,4
3,3,4,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0,12,48,1,335,4
4,4,5,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0,12,48,1,335,4


In [32]:
date_eng3 = date_eng2.iloc[: , 2:]
date_eng3.head()

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week
0,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0,12,48,1,335,4
1,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0,12,48,1,335,4
2,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0,12,48,1,335,4
3,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0,12,48,1,335,4
4,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0,12,48,1,335,4


In [33]:
date_eng3.describe()

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,11.500000,704.602055,12.882922,58.226256,1.724909,14368.257991,4.073813,0.569111,0.148687,0.750685,6.526027,26.558904,15.720548,183.000000,3.002740
std,6.922582,644.997468,11.944825,20.362413,1.036300,6082.987120,13.060369,0.868746,1.128193,4.367462,3.448048,15.030690,8.796749,105.372043,1.998056
min,0.000000,0.000000,-17.800000,0.000000,0.000000,270.000000,-30.600000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,5.750000,191.000000,3.500000,42.000000,0.900000,9400.000000,-4.700000,0.000000,0.000000,0.000000,4.000000,14.000000,8.000000,92.000000,1.000000
50%,11.500000,504.500000,13.700000,57.000000,1.500000,16980.000000,5.100000,0.010000,0.000000,0.000000,7.000000,27.000000,16.000000,183.000000,3.000000
75%,17.250000,1065.250000,22.500000,74.000000,2.300000,20000.000000,14.800000,0.930000,0.000000,0.000000,10.000000,40.000000,23.000000,274.000000,5.000000
max,23.000000,3556.000000,39.400000,98.000000,7.400000,20000.000000,27.200000,3.520000,35.000000,88.000000,12.000000,52.000000,31.000000,365.000000,6.000000


In [35]:
_ , p_value = calculate_bartlett_sphericity(date_eng3)
p_value

0.0

In [37]:
_, kmo_score = calculate_kmo(date_eng3)
kmo_score 

0.44929815032033743

In [53]:
date_eng21 = pd.read_csv('ft_eng2.csv')
date_eng21

,Unnamed: 0,X1,Unnamed: 0.1,datetime,date,hour,season,holiday,open,rent_count,...,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week,dayName_of_week,month_name,season_no
0,1,0,1,2017-12-01 00:00:00,2017-12-01,0,Winter,No Holiday,Yes,254,...,0.0,0,12,48,1,335,4,Friday,Dec,4
1,2,1,2,2017-12-01 01:00:00,2017-12-01,1,Winter,No Holiday,Yes,204,...,0.0,0,12,48,1,335,4,Friday,Dec,4
2,3,2,3,2017-12-01 02:00:00,2017-12-01,2,Winter,No Holiday,Yes,173,...,0.0,0,12,48,1,335,4,Friday,Dec,4
3,4,3,4,2017-12-01 03:00:00,2017-12-01,3,Winter,No Holiday,Yes,107,...,0.0,0,12,48,1,335,4,Friday,Dec,4
4,5,4,5,2017-12-01 04:00:00,2017-12-01,4,Winter,No Holiday,Yes,78,...,0.0,0,12,48,1,335,4,Friday,Dec,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756,8755,8756,2018-11-30 19:00:00,2018-11-30,19,Autumn,No Holiday,Yes,1003,...,0.0,0,11,48,30,334,4,Friday,Nov,3
8756,8757,8756,8757,2018-11-30 20:00:00,2018-11-30,20,Autumn,No Holiday,Yes,764,...,0.0,0,11,48,30,334,4,Friday,Nov,3
8757,8758,8757,8758,2018-11-30 21:00:00,2018-11-30,21,Autumn,No Holiday,Yes,694,...,0.0,0,11,48,30,334,4,Friday,Nov,3
8758,8759,8758,8759,2018-11-30 22:00:00,2018-11-30,22,Autumn,No Holiday,Yes,712,...,0.0,0,11,48,30,334,4,Friday,Nov,3


In [54]:
date_eng21.describe()

,Unnamed: 0,X1,Unnamed: 0.1,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week,season_no
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,4380.500000,4379.500000,4380.500000,11.500000,704.602055,12.882922,58.226256,1.724909,14368.257991,4.073813,0.569111,0.148687,0.750685,6.526027,26.558904,15.720548,183.000000,3.002740,2.490411
std,2528.938512,2528.938512,2528.938512,6.922582,644.997468,11.944825,20.362413,1.036300,6082.987120,13.060369,0.868746,1.128193,4.367462,3.448048,15.030690,8.796749,105.372043,1.998056,1.116831
min,1.000000,0.000000,1.000000,0.000000,0.000000,-17.800000,0.000000,0.000000,270.000000,-30.600000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,2190.750000,2189.750000,2190.750000,5.750000,191.000000,3.500000,42.000000,0.900000,9400.000000,-4.700000,0.000000,0.000000,0.000000,4.000000,14.000000,8.000000,92.000000,1.000000,1.000000
50%,4380.500000,4379.500000,4380.500000,11.500000,504.500000,13.700000,57.000000,1.500000,16980.000000,5.100000,0.010000,0.000000,0.000000,7.000000,27.000000,16.000000,183.000000,3.000000,2.000000
75%,6570.250000,6569.250000,6570.250000,17.250000,1065.250000,22.500000,74.000000,2.300000,20000.000000,14.800000,0.930000,0.000000,0.000000,10.000000,40.000000,23.000000,274.000000,5.000000,3.000000
max,8760.000000,8759.000000,8760.000000,23.000000,3556.000000,39.400000,98.000000,7.400000,20000.000000,27.200000,3.520000,35.000000,88.000000,12.000000,52.000000,31.000000,365.000000,6.000000,4.000000


In [55]:
date_eng21_colnames = list(date_eng21.columns)
date_eng21_num_colnames = list(date_eng21.describe().columns)
diff_names2 = []
for i in date_eng21_colnames:
	if i not in date_eng21_num_colnames:
		diff_names2.append(i)
diff_names2

['datetime',
 'date',
 'season',
 'holiday',
 'open',
 'dayName_of_week',
 'month_name']

In [56]:
date_eng22 = date_eng21.copy()
date_eng22.drop(columns = diff_names2,inplace=True)
date_eng22.head()

,Unnamed: 0,X1,Unnamed: 0.1,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week,season_no
0,1,0,1,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
1,2,1,2,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
2,3,2,3,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0,12,48,1,335,4,4
3,4,3,4,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
4,5,4,5,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0,12,48,1,335,4,4


In [57]:
date_eng23 = date_eng22.iloc[: , 3:]
date_eng23.head()

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall,month,week,day_of_month,day_of_year,day_of_week,season_no
0,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
1,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
2,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0,12,48,1,335,4,4
3,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0,12,48,1,335,4,4
4,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0,12,48,1,335,4,4


In [59]:
_ , p_value = calculate_bartlett_sphericity(date_eng23)
p_value


0.0

In [60]:
_, kmo_score = calculate_kmo(date_eng23)
kmo_score 

0.3758310591819197